In [257]:
import collections
import numpy as np
import pandas as pd
from scipy import special
from scipy import stats

In [47]:
LABELS = collections.OrderedDict([
    ('time', 'Time (weeks), $i$'),
    ('failures', 'Failures, $y_i$'),
    ('temperature', 'Temperature, $x_i$'),
])

failure_time_data = pd.DataFrame(collections.OrderedDict([
    ('time', ['${}$'.format(week) for week in range(1,13)] + ['$>12$']),
    ('failures', [210, 108, 58, 40, 17, 10, 7, 6, 5, 4, 2, 3, 15]),
    ('temperature', [24, 26, 24, 26, 25, 22, 23, 20, 21, 18, 17, 20, None]),
]))

with open('failure_time_data.tex', 'w') as f:
    f.write(
        pd.DataFrame(collections.OrderedDict(
            [(LABELS[i], value) for i, value in failure_time_data.items()])).to_latex(
            escape=False, index=False, na_rep=''))

failure_time_data

,time,failures,temperature
0,$1$,210,24.0
1,$2$,108,26.0
2,$3$,58,24.0
3,$4$,40,26.0
4,$5$,17,25.0
5,$6$,10,22.0
6,$7$,7,23.0
7,$8$,6,20.0
8,$9$,5,21.0
9,$10$,4,18.0


# Geometric Model

In [223]:
def score(y, p):
    y = np.asarray(y)
    N = len(y) - 1
    t = np.arange(1, len(y))
    return -N*y[-1]/(1-p) + np.sum(y[:-1]*(1/p - (t-1)/(1-p)))

def mle(y):
    y = np.asarray(y)
    return np.sum(y[:-1])/(y[-1]*(len(y) - 1) + np.sum(np.arange(1, len(y))*y[:-1]))

score(failure_time_data['failures'], mle(failure_time_data['failures']))
mle(failure_time_data['failures'])

0.35471698113207545

In [239]:
def fisher_information(y, p):
    N = len(y) - 1
    n = np.sum(y)
    return n*(1 - (1-p)**N)/(p*p*(1-p))

1/fisher_information(failure_time_data['failures'], mle(failure_time_data['failures']))

0.00016828364537217432

In [254]:
stats.norm.interval(
    0.95,
    loc=mle(failure_time_data['failures']),
    scale=np.sqrt(1/fisher_information(failure_time_data['failures'], mle(failure_time_data['failures']))))

(0.32929150780524236, 0.38014245445890854)

In [253]:
np.sqrt(0.00016828364537217432)*stats.norm.ppf(0.025) + 0.35471698113207545

0.32929150780524236

In [125]:
tmp = mle(failure_time_data['failures'])
tmp

np.sum((1 - tmp)**np.arange(13))

2.809667566181416

In [128]:
(1 - (1-tmp)**13)/tmp

2.8096675661814157

In [219]:
np.sum((np.arange(1,13) - 1)*((1-tmp)**(np.arange(1,13) - 1)))

4.9254021801953245

In [208]:
np.sum(np.arange(4)*tmp**np.arange(4))*(1-tmp)

0.4776779356290193

In [218]:
(tmp - tmp**4)/(1-tmp) - 3*tmp**4

0.4776779356290193

In [205]:
tmp/(1-tmp)**2

0.8518860504086725

In [221]:
(1 - tmp - (1-tmp)**12)/tmp**2 - 11*((1-tmp)**12)/tmp

4.925402180195324

In [188]:
tmp2 = 1-tmp
((tmp2 - tmp2**11)/tmp - tmp2**11)/tmp

5.041488246900949

In [255]:
#np.arange(1, len(failure_time_data))*
np.sum(np.arange(1, len(failure_time_data))*failure_time_data['failures'].values[:-1])

1145

In [292]:
alpha = 2
beta = 4

special.beta(alpha,beta)

0.05

In [296]:
s = np.arange(1000)
np.sum(special.beta(alpha + 1, beta + s))

0.04999900893936691

In [290]:
np.sum(special.gamma(beta + s)/special.gamma(alpha+beta+s+1))

0.024954212454212452

In [293]:
special.gamma(beta)/special.gamma(alpha+beta)/alpha

0.025

In [307]:
s = np.arange(100)
np.sum(special.beta(alpha + 1, beta + s)/special.beta(alpha, beta))

0.9981684981684983

In [306]:
s = np.arange(100)
np.sum(alpha*special.gamma(alpha + beta)*special.gamma(beta+s)/special.gamma(beta)/special.gamma(alpha+beta+s + 1))

0.9981684981684983